In [1]:
from classes import Body
from classes import NBodySimulation

import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt

In [11]:
# initializing bodies
# body_n = Body(parameters)

body_1 = Body(mass = 1, position = [1, 1, 1], velocity = [1, 1, 1])
body_2 = Body(mass = 2, position = [3, 5, 2], velocity = [1, 2, 1])

# package bodies together for NBodySimulation
bodies = [body_1, body_2]

In [8]:
# run simulation
simulation = NBodySimulation(bodies)